### SPE for strax
- Estimation of the SPE acceptance using low-intensity LED runs and 'blank' runs that are triggered externally just as for LED runs but with no light emitted;
- Starting from Evan's work (https://xe1t-wiki.lngs.infn.it/doku.php?id=xenon:saldanha:xe1t:single_photoelectron_amplitude), I am trying to implement using strax environment;
- Useful for strax: https://github.com/XENONnT/workshop_strax_chicago/tree/master/projects/LED_calibration;
- Strax documentation: https://straxen.readthedocs.io/en/latest/.

In [1]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
pd.options.display.max_colwidth = 100
import strax
from scipy.optimize import curve_fit
from scipy import stats

import straxen
st = straxen.contexts.strax_workshop_dali()

In [2]:
%run '/home/gvolta/Desktop/SPE/Function.ipynb'

## Datasets

In [3]:
#runs = st.select_runs(available='raw_records', run_mode='LED*', include_tags=['spe_topring', 'spe_topbulk', 'spe_bottom'])
runs = st.select_runs(run_mode='LED*')
runs

Checking data availability: 100%|██████████| 5/5 [00:07<00:00,  1.33s/it]


,end,mode,name,number,reader.ini.name,start,tags,trigger.events_built,tags.name,records_available,raw_records_available,event_info_available,peaks_available,events_available
41,2018-02-19 10:00:16+00:00,LED_3mus_stable,180219_0952,16979,LED_3mus_stable,2018-02-19 09:52:13+00:00,"gain_step0,_sciencerun2_candidate",196157.0,NaN,False,True,False,False,False
42,2018-02-19 10:09:44+00:00,LED_3mus_stable,180219_1001,16980,LED_3mus_stable,2018-02-19 10:01:41+00:00,"gain_step1,_sciencerun2_candidate",196207.0,NaN,False,True,False,False,False
43,2018-02-19 10:19:09+00:00,LED_3mus_stable,180219_1011,16981,LED_3mus_stable,2018-02-19 10:11:06+00:00,"gain_step2,_sciencerun2_candidate",194686.0,NaN,False,True,False,False,False
44,2018-02-19 10:29:22+00:00,LED_3mus_stable,180219_1021,16982,LED_3mus_stable,2018-02-19 10:21:18+00:00,"gain_step3,_sciencerun2_candidate",196373.0,NaN,False,True,False,False,False
45,2018-02-19 10:39:05+00:00,LED_3mus_stable,180219_1030,16983,LED_3mus_stable,2018-02-19 10:31:02+00:00,"gain_step4,_sciencerun2_candidate",195938.0,NaN,False,True,False,False,False
46,2018-02-19 10:58:02+00:00,LED_3mus_stable,180219_1049,16985,LED_3mus_stable,2018-02-19 10:50:00+00:00,"spe_topbulk,_sciencerun2_candidate",194366.0,NaN,False,True,False,False,False
47,2018-02-19 11:07:24+00:00,LED_3mus_stable,180219_1059,16986,LED_3mus_stable,2018-02-19 10:59:21+00:00,"spe_topring,_sciencerun2_candidate",193050.0,NaN,False,True,False,False,False
48,2018-02-19 11:15:31+00:00,LED_7mus_stable,180219_1110,16987,LED_7mus_stable,2018-02-19 11:10:28+00:00,"Afterpulse,_sciencerun2_candidate",46116.0,NaN,False,True,False,False,False
49,2018-02-19 11:31:50+00:00,LED_7mus_stable,180219_1116,16988,LED_7mus_stable,2018-02-19 11:16:48+00:00,"Afterpulse,_sciencerun2_candidate",137431.0,NaN,False,True,False,False,False


In [4]:
st.data_info('raw_records')

,Field name,Data type,Comment
0,channel,int16,Channel/PMT number
1,dt,int16,Time resolution in ns
2,time,int64,Start time of the interval (ns since unix epoch)
3,length,int32,Length of the interval in samples
4,area,int32,Integral in ADC x samples
5,pulse_length,int32,Length of pulse to which the record belongs (without zero-padding)
6,record_i,int16,Fragment number in the pulse
7,baseline,float32,Baseline in ADC counts. data = int(baseline) - data_orig
8,reduction_level,uint8,Level of data reduction applied (strax.ReductionLevel enum)
9,data,"('<i2', (110,))",Waveform data in ADC counts above baseline


- Selection of LED data: 180219_1049, 180219_1059;
- Just the first 30 sec for each event.

In [5]:
#data_rr_LED = st.get_array(['180219_1049', '180219_1059'], 'raw_records', seconds_range=(0, 30))
#np.save('data_rr_LED', data_rr_LED)
data_rr_LED = np.load('data_rr_LED.npy')

In [6]:
#data_rr_noise_0 = st.get_array(['180219_0952'], 'raw_records', seconds_range=(0, 30))
#np.save('data_rr_noise_0', data_rr_noise_0)
data_rr_noise_0 = np.load('data_rr_noise_0.npy')

In [7]:
#data_rr_noise_1 = st.get_array(['180219_1030'], 'raw_records', seconds_range=(0, 20))
#np.save('data_rr_noise_1', data_rr_noise_1)
data_rr_noise_1 = np.load('data_rr_noise_1.npy')

### LED waveform

In [ ]:
plt.figure(figsize=(10,12))

plt.subplot(211)
PMT_25_rr_LED = data_rr_LED[data_rr_LED['channel']==25]
plot_peak_2(PMT_25_rr_LED[0], label=pd.to_datetime(PMT_25_rr_LED[0]['time']))
plt.title('channel ' + str(PMT_25_rr_LED['channel'][0]))

plt.subplot(212)
PMT_200_rr_LED = data_rr_LED[data_rr_LED['channel']==200]
plot_peak_2(PMT_200_rr_LED[0], label=pd.to_datetime(PMT_200_rr_LED[0]['time']))
plt.title('channel ' + str(PMT_200_rr_LED['channel'][0]))

plt.show()
plt.tight_layout()

### Noise waveform

In [ ]:
plt.figure(figsize=(10,12))

plt.subplot(211)
PMT_25_rr_noise_0 = data_rr_noise_0[data_rr_noise_0['channel']==25]
plot_peak_2(PMT_25_rr_noise_0[0], label=pd.to_datetime(PMT_25_rr_noise_0[0]['time']))
plt.title('channel ' + str(PMT_25_rr_noise_0['channel'][0]))

plt.subplot(212)
PMT_200_rr_noise_1 = data_rr_noise_1[data_rr_noise_1['channel']==200]
plot_peak_2(PMT_200_rr_noise_1[0], label=pd.to_datetime(PMT_200_rr_noise_1[0]['time']))
plt.title('channel ' + str(PMT_200_rr_noise_1['channel'][0]))

plt.show()
plt.tight_layout()

### Rough SPE Amplitude determination and LED Window Identification
- For each event, and on each channel, we determine the maximum amplitude of the waveform, and then add it to the amplitude spectrum for the corresponding channel;
- Once the rough amplitude range of single photoelectrons is determined, we go back and search for waveforms with maximum values falling in this single pe range. 

In [ ]:
info = {'mean': [], 'sig': [], 'Norm': [], 'Const': [], 'pmt': []}
d2 = pd.DataFrame({'channel': [], 'idx_LED': []}) 
n_channel_s = np.arange(0, 254, 1)
pmts_rejected = []
x = np.arange(0,600,1)

for n_channel in tqdm(n_channel_s):
    #if n_channel in pmts_rejected:
    #     continue   
    wf_tmp = data_rr_LED[data_rr_LED['channel'] == n_channel]
    amp_wf = []
    idx_amp_wf = []
    idx_LED = []
    for wf_ in wf_tmp:
        amp_wf.append(np.max(wf_['data']))
        idx_amp_wf.append(np.argmax(wf_['data']))
    amp_wf = np.array(amp_wf)
    idx_amp_wf = np.array(idx_amp_wf)

    #hist, xbins, _ = plt.hist(amp_wf, bins=100, range=(0,200))
    hist, xbins = np.histogram(amp_wf, bins=100, range=(0,200))
    xbins_center = np.array([0.5*(xbins[i]+xbins[i+1]) for i in range(len(xbins)-1)])
    #plt.xlabel('amp (ADC counts)')
    mask = (xbins_center > 30) & (xbins_center < 200)
    popt, pcov = curve_fit(gaus, xbins_center[mask], hist[mask], p0=[10, 60, 20, 0], maxfev=int(1e6))
    #hist.plot()
    #plt.plot(x, gaus(x, *popt), 'r-', label='Mean: %.1f\nStd: %.1f' %(popt[1], popt[2]))
    #plt.title('Channel %d' %n_channel)
    #plt.yscale('log')
    #plt.legend(loc='best')
    #plt.show()
        
    N, mean, sig, c = popt[0], popt[1], popt[2], popt[3]
    #print(mean, sig)
    info['mean'].append(popt[1])
    info['sig'].append(popt[2])
    info['Norm'].append(popt[0])
    info['Const'].append(popt[3])
    info['pmt'].append(n_channel)
    
    d1 = pd.DataFrame({'channel': [], 'idx_LED': []})
    mask = (amp_wf < mean + sig) & (amp_wf > mean - sig)
    idx_LED = idx_amp_wf[mask]
    if len(idx_LED)==0:
        d1['channel'] = n_channel
        d1['idx_LED'] = np.nan
    else:
        d1['idx_LED'] = idx_LED
        d1['channel'] = np.ones_like(idx_LED) * n_channel
    d2 = d2.append(d1, ignore_index=True)
    del d1, amp_wf, idx_amp_wf, idx_LED

In [ ]:
fig = plt.figure(figsize=(16,6))
plt.subplot(121)
hist2d, xbins, ybins, _ = plt.hist2d(x = d2['channel'], y = d2['idx_LED'], bins=(254,600), range=((0,254),(0,600)), 
                                               cmap=plt.cm.plasma, norm=matplotlib.colors.LogNorm(), cmin = 1,alpha = 1)
plt.colorbar(label='Number of events')
plt.xlabel('PMT')
plt.ylabel('sample')

plt.subplot(122)
y_hist, y_bins, _ = plt.hist(d2['idx_LED'], bins=600, range=(0,600), histtype='step', color='black')
plt.yscale('log')
plt.xlim(0,600)
mean = d2['idx_LED'].mean()
std = d2['idx_LED'].std()
window = [int(mean-0.5*std),int(mean+0.5*std)]
print('mean: ',mean,'\t standard deviation: ', std)
plt.axvspan(*window, color='grey', alpha=0.2)
plt.title('LED window: '+str(mean-0.5*std)+' - '+str(mean+0.5*std))
plt.xlabel('Number of events')
plt.ylabel('sample')

plt.show()
plt.tight_layout()

### Noise Matching
- We would like to try and determine the single photoelectron spectrum down to as low amplitude values as possible, and so we will attempt to subtract away the noise contribution;
- We will do this noise subtraction by taking residual between the LED and noise runs for each configuration. Since the noise run uses the same trigger logic, etc. as the LED run, the amplitude spectrum in the LED window determined above should represent the amplitude spectrum of the noise.

In [ ]:
df_LED = pd.DataFrame({'Channel': [], 'Amplitude': [], 'Time': []})
df_noise = pd.DataFrame({'Channel': [], 'Amplitude': [], 'Time': []}) 
n_channel_s = np.arange(0, 20, 1)
pmts_rejected = []
x = np.arange(0,600,1)

for n_channel in tqdm(n_channel_s):
    #if n_channel in pmts_rejected:
    #     continue   
    LED_tmp = data_rr_LED[data_rr_LED['channel'] == n_channel]
    noise_tmp = data_rr_noise_0[data_rr_noise_0['channel'] == n_channel]
    
    d1_LED = pd.DataFrame({'Channel': [], 'Amplitude': [], 'Time': []})
    d1_noise = pd.DataFrame({'Channel': [], 'Amplitude': [], 'Time': []})
    
    amp_LED = []
    time_amp_LED = []
    amp_noise = []
    time_amp_noise = []
    
    for LED_ in LED_tmp:
        amp_LED.append(np.max(LED_['data'][window[0]:window[1]]))
        time_amp_LED.append(np.argmax(LED_['data'][window[0]:window[1]]))
    for noise_ in noise_tmp:
        amp_noise.append(np.max(noise_['data'][window[0]:window[1]]))
        time_amp_noise.append(np.argmax(noise_['data'][window[0]:window[1]]))
        
    d1_LED['Channel'] = np.ones_like(amp_LED) * n_channel
    d1_noise['Channel'] = np.ones_like(amp_noise) * n_channel
    
    d1_LED['Amplitude'] = np.array(amp_LED)
    d1_noise['Amplitude'] = np.array(amp_noise)
    
    d1_LED['Time'] = np.array(time_amp_LED)
    d1_noise['Time'] = np.array(time_amp_noise)
    
    df_LED = df_LED.append(d1_LED, ignore_index=True)
    df_noise = df_noise.append(d1_noise, ignore_index=True)    
    
    
    fig = plt.figure(figsize=(30,8))
    
    #amp_LED = df_LED[df_LED['Channel'] == n_channel]
    #amp_noise = df_noise[df_noise['Channel'] == n_channel]
    LED_spectrum, bins_1 = np.histogram(d1_LED['Amplitude'], bins=150, range=(0,300))
    noise_spectrum, bins_2 = np.histogram(d1_noise['Amplitude'], bins=150, range=(0,300))
    
    plt.subplot(141)
    #plt.hist(d1_LED['Amplitude'], bins=100, range=(0,200), label='LED')
    #plt.hist(d1_noise['Amplitude'], bins=100, range=(0,200), label='noise')
    plt.plot(bins_1[:len(bins_1)-1], LED_spectrum, color='k', label='LED')
    plt.plot(bins_2[:len(bins_2)-1], noise_spectrum, color='r', label='noise')
    plt.xlabel('amp (ADC counts)')
    plt.title('Channel %d' %n_channel)
    plt.yscale('log')
    plt.legend(loc='best')
    plt.xlim(0)
    
    ADC_correction = 7
    tmp_1 = d1_LED[d1_LED['Amplitude'] < ADC_correction]
    tmp_2 = d1_noise[d1_noise['Amplitude'] < ADC_correction]
    scaling_coeff = tmp_1['Amplitude'].sum()/tmp_2['Amplitude'].sum()
    noise_spectrum_2 = noise_spectrum*scaling_coeff
 
    plt.subplot(142)
    plt.errorbar(x = bins_1[:len(bins_1)-1], y = LED_spectrum, yerr = np.sqrt(LED_spectrum), fmt='k+', label='LED')
    plt.errorbar(x = bins_2[:len(bins_2)-1],  y = noise_spectrum_2, yerr = np.sqrt(noise_spectrum_2), fmt='r+', label='noise')
    plt.vlines(x=ADC_correction, ymin=0, ymax=1e4, colors='k', linestyles='dashed')
    plt.xlabel('amp (ADC counts)')
    plt.title('Channel %d - ZOOM' %n_channel)
    plt.yscale('log')
    plt.legend(loc='best')
    plt.xlim(0,50)
    

    plt.subplot(143)
    #hist_LED, xbins_LED = np.histogram(d1_LED['Amplitude'], bins=50, range=(0,50))
    #hist_noise, xbins_noise = np.histogram(d2_noise, bins=50, range=(0,50))
    x_center = [(bins_1[i]+bins_1[i+1])*0.5 for i in range(len(bins_1)-1)]
    diff = abs(LED_spectrum - noise_spectrum*coeff)
    sigma_diff = np.sqrt(LED_spectrum - noise_spectrum*coeff)
    plt.errorbar(x = x_center, y=diff, yerr = sigma_diff, fmt='b+' )
    #hist_diff, xbins_diff, _ = plt.hist(diff, bins=50, range=(0,50), label='Difference', histtype='step')
    plt.xlabel('amp (ADC counts)')
    plt.title('Channel %d' %n_channel)
    plt.legend(loc='best')
    
    
    plt.subplot(144)
    res =  1. - np.cumsum(diff)/np.sum(diff)
    plt.plot(x_center, res)
    plt.title('Acceptance')
    plt.xlim(0)
    plt.xlabel('amp (ADC counts)')
    plt.title('Channel %d' %n_channel)
    plt.legend(loc='best')
        
    plt.show()
    
    del d1_LED, d1_noise, amp_LED, amp_noise, time_amp_LED, time_amp_noise, tmp_1, tmp_2, x_center, diff, sigma_diff, res